In [1]:
1e-4

0.0001

In [1]:
import os
import math
import pandas as pd
import numpy as np
from pathlib import Path
import shutil
import argparse
from typing import List, Union
import matplotlib.pyplot as plt
import yaml
from datetime import datetime
import logging


import torch
from torch import Tensor, nn
from torch.types import Device, _size
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import init
from torch.utils.data import Dataset
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from collections import OrderedDict

from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
from configs.config import configs

from models.resnet1D_Copy import *
from models.transformer_encoder import transformer_classifier

2024-06-05 14:38:10.088944: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 14:38:10.107772: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 14:38:10.388203: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Initilization

In [ ]:
# Init logging
import logging

logger = logging.getLogger(__name__)  # Use the current module's name
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)
logger.addHandler(handler)
acc_example = 0.95  # Replace with your actual accuracy calculation
logger.info(f"Current accuracy: %{acc_example}")  # Log as info
# logger.debug("Current accuracy: %.2f", accuracy)  # Log as info

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("config_file", metavar="FILE", help="config file")
# parser.add_argument('--run-dir', metavar='DIR', help='run directory')
# parser.add_argument('--pdb', action='store_true', help='pdb')
args = parser.parse_args(args=['configs/eeg_torch.yml'])
# args, opts = parser.parse_known_args()
# f = 'configs/eeg_pt.yml'
with open(args.config_file, 'r') as file:
    Configs = yaml.safe_load(file)

In [ ]:
class customDataset(Dataset):
    def __init__(self, data_dir:str, label_dir:str, label_dict:dict, transform=None):
#         self.annotations = pd.read_csv(label_dir)
        self.data_dir = data_dir   # './data/origin_csv/train'
        self.label_dir = label_dir  # './data/train_label.csv'
        self.transform = transform   
        self.files = os.listdir(self.data_dir)  # train_dataset.files[24580]='aaaaanrb_s002_t000_7.csv', abnormal
                                                # train_dataset.files[2]='aaaaaetn_s002_t000_6.csv', normal
        self.annotations = pd.read_csv(self.label_dir)   # (28434, 2)   aaaaaool_s004_t001_1.csv	abnormal
        self.label_dict = label_dict     #   {'normal': 0, 'abnormal': 1}
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, index):
        data_path = os.path.join(self.data_dir, self.files[index])
        data = pd.read_csv(data_path)
        data = torch.tensor(data.values, dtype=torch.float32)
        file_name = self.files[index]
        label = self.annotations.loc[(self.annotations['csv_file']==file_name)]
        label = label.iloc[0,1]
        label_tensor = torch.tensor(int(self.label_dict[label]))
        
        if self.transform:
            data = self.transform(data)
            
        return (data.t(), label_tensor, file_name)

In [ ]:
train_data_dir = Configs['dataset']['train_data_dir']
train_label_dir = Configs['dataset']['train_label_dir']

# val_data_dir = Configs['dataset']['val_data_dir']
# val_label_dir = Configs['dataset']['val_label_dir']

label_dict = Configs['dataset']['classes']
train_dataset = customDataset(data_dir=train_data_dir,
                              label_dir=train_label_dir,
                              label_dict=label_dict)

In [ ]:
class model(nn.Module):
    def __init__(self, input_size: int, n_channels: int, model_hyp: dict, classes: int):
        super(model, self).__init__()
        self.ae = resnet18()
        self.transformer_encoder = transformer_classifier(input_size, n_channels, model_hyp, classes)

    def forward(self, x):
        z = self.ae(x)
        z = self.transformer_encoder(z)
        return z

In [ ]:
transformer_encoder = transformer_classifier(Configs['input_size'], 
                                             Configs['n_channels'], 
                                             Configs['model'], 
                                             len(Configs['dataset']['classes'])).to('cuda')

In [ ]:
train_data_dir = Configs['dataset']['train_data_dir']
train_label_dir = Configs['dataset']['train_label_dir']

val_data_dir = Configs['dataset']['val_data_dir']
val_label_dir = Configs['dataset']['val_label_dir']

label_dict = Configs['dataset']['classes']
train_dataset = customDataset(data_dir=train_data_dir,
                              label_dir=train_label_dir,
                              label_dict=label_dict)
val_dataset = customDataset(data_dir=val_data_dir,
                            label_dir=val_label_dir,
                            label_dict=label_dict)

train_loader = DataLoader(dataset=train_dataset, batch_size=Configs['train']['batch_size'],
                              shuffle=True, num_workers=16, pin_memory=True)

eval_loader = DataLoader(dataset=val_dataset, num_workers=16, shuffle=True, pin_memory=True)

classifier = model(input_size=Configs['input_size'],
                                    n_channels = Configs['n_channels'],
                                    model_hyp=Configs['model'],
                                    classes=len(Configs['dataset']['classes'])).to('cuda')
optimizer = torch.optim.Adam(classifier.parameters(),betas=(0.98,0.98),lr=Configs['optimizer']['init_lr'])
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter(Configs['tensorboard']['runs_dir']+'train_board')    # Initilize tensorflow

In [ ]:
warmup_steps = Configs['train']['warmup_steps']
warmup_step = 0
min_loss = 0.6
val_loss = 0
correct = 0
total = 0
val_accuracy = 0
ae = resnet18().to('cuda')

dataset_iter = iter(train_loader)
(data,target,_) = next(dataset_iter)
# for batch_index, (data,target,_) in enumerate(train_loader, 0):
#     if warmup_step < warmup_steps:
# #     for batch_index, data in enumerate(train_loader, 0):
#         data, target = data.to('cuda'), target.to('cuda')
#         y = ae(data)

In [ ]:
if Configs['warmup']==1:
    ### Warmup training
    warmup_steps = Configs['train']['warmup_steps']
    warmup_step = 0
    min_loss = 0.6
    val_loss = 0
    correct = 0
    total = 0
    val_accuracy = 0
    model_params = {
        'model_state_dict': classifier.state_dict(),
        'min_loss': min_loss,
        'val_accuracy': val_accuracy
    }
    while warmup_step < warmup_steps:
        for batch_index, (data,target,_) in enumerate(train_loader, 0):
            if warmup_step < warmup_steps:
        #     for batch_index, data in enumerate(train_loader, 0):
                data, target = data.to('cuda'), target.to('cuda')
                y = classifier(data)
        #         logger.debug(f"y size:{y.shape}, tatget size{target.shape}")
                warmup_loss = criterion(y, target)
                optimizer.zero_grad()
                warmup_loss.backward()
                optimizer.step()
        #         logger.info(f'Epoch: {epoch+1}, Train Loss: {train_loss}')
                logger.info(f"Warmup Step: {warmup_step}, Warmup Loss: {warmup_loss}")
                writer.add_scalar('Warmup Loss', warmup_loss, global_step=warmup_step)
                warmup_step += 1

            if warmup_loss < min_loss:
                with torch.no_grad():
                    for batch_index, (data,target,_) in enumerate(eval_loader, 0):
                        data, target = data.to('cuda'), target.to('cuda')
                        outputs = classifier(data)
                        loss = criterion(outputs, target)
                        val_loss += loss.item()
                        _, predicted = torch.max(outputs, 1)
                        total += target.size(0)  # Total number of samples
                        correct += (predicted == target).sum().item()  # Count correct predictions

                        val_loss /= len(eval_loader)
                        val_accuracy = 100 * correct / total
                    logger.info(f'Warmup Step: {warmup_step}, Warmup Loss: {warmup_loss}, Val Loss: {val_loss}, Accuracy:{val_accuracy:.2f}%')
                min_loss = warmup_loss
                torch.save(model_params,
                           Configs['checkpoint']['checkpoint_dir']+'inte_transformer_params_best.pth')


        if warmup_step%5==0:
            torch.save(model_params, 
                       Configs['checkpoint']['checkpoint_dir']+'inte_transformer_params_last.pth')
        logger.info(f'min_loss: {min_loss}, Accuracy:{val_accuracy:.2f}%')

### Dataset

In [ ]:
class customDataset(Dataset):
    def __init__(self, data_dir:str, label_dir:str, label_dict:dict, transform=None):
#         self.annotations = pd.read_csv(label_dir)
        self.data_dir = data_dir   # './data/origin_csv/train'
        self.label_dir = label_dir
        self.transform = transform
        self.files = os.listdir(self.data_dir)
        self.annotations = pd.read_csv(self.label_dir)
        self.label_dict = label_dict
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, index):
        data_path = os.path.join(self.data_dir, self.files[index])
        data = pd.read_csv(data_path)
        data = torch.tensor(data.values, dtype=torch.float32)
        file_name = self.files[index]
        
        label = torch.tensor(int(self.label_dict[self.annotations.iloc[index,1]]))
        
        if self.transform:
            data = self.transform(data)
            
#         return (data.t(), label, file_name)
        return (data.t(), label, file_name)

In [ ]:

# label_dic = {'normal':0, 'abnormal':1}

    
# transform = transforms.Compose([
#     transforms.MinMaxScaler(feature_range=(0, 1)),
#     transforms.ToTensor(),
# ])
# combined_dataset = ConcatDataset([train_dataset, eval_dataset])

### Define model

In [ ]:
### Learning rate update policy
def poly_lr_scheduler(optimizer, init_lr, iter, lr_decay_iter=1,
                      max_iter=0, power=0.9):
    """Polynomial decay of learning rate
        :param init_lr is base learning rate
        :param iter is a current iteration
        :param lr_decay_iter how frequently decay occurs, default is 1
        :param max_iter is number of maximum iterations
        :param power is a polymomial power
    """
    if max_iter == 0:
        raise Exception("MAX ITERATION CANNOT BE ZERO!")
    if iter % lr_decay_iter or iter > max_iter:
        return optimizer
    lr = init_lr * (1 - iter / max_iter) ** power
    logger.info(f'lr=: {lr}')
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

In [ ]:
def train(Configs:dict):
    train_data_dir = Configs['dataset']['train_data_dir']
    train_label_dir = Configs['dataset']['train_label_dir']

    val_data_dir = Configs['dataset']['val_data_dir']
    val_label_dir = Configs['dataset']['val_label_dir']

    label_dict = Configs['dataset']['classes']
    train_dataset = customDataset(data_dir=train_data_dir,
                                  label_dir=train_label_dir,
                                  label_dict=label_dict)
    val_dataset = customDataset(data_dir=val_data_dir,
                                label_dir=val_label_dir,
                                label_dict=label_dict)
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=Configs['train']['batch_size'],
                                  shuffle=True, num_workers=16, pin_memory=True)

    eval_loader = DataLoader(dataset=val_dataset, num_workers=16, shuffle=True, pin_memory=True)

    classifier = transformer_classifier(input_size=Configs['input_size'], 
                                        n_channels = Configs['n_channels'],
                                        model_hyp=Configs['model'],
                                        classes=len(Configs['dataset']['classes'])).to('cuda')
    optimizer = torch.optim.Adam(classifier.parameters(),betas=(0.9,0.9),lr=Configs['optimizer']['init_lr'])
    criterion = nn.CrossEntropyLoss()
    writer = SummaryWriter(Configs['tensorboard']['runs_dir']+'train_board')    # Initilize tensorflow
    
    ### Warmup training
    warmup_steps = Configs['train']['warmup_steps']
    warmup_step = 0
    min_loss = 1
    while warmup_step < warmup_steps:
        for batch_index, (data,target,_) in enumerate(train_loader, 0):
            if warmup_step < warmup_steps:
        #     for batch_index, data in enumerate(train_loader, 0):
                data, target = data.to('cuda'), target.to('cuda')
                y = classifier(data)
        #         logger.debug(f"y size:{y.shape}, tatget size{target.shape}")
                warmup_loss = criterion(y, target)
                optimizer.zero_grad()
                warmup_loss.backward()
                optimizer.step()
        #         logger.info(f'Epoch: {epoch+1}, Train Loss: {train_loss}')
                logger.info(f"Warmup Step: {warmup_step}, Warmup Loss: {warmup_loss}")
                writer.add_scalar('Warmup Loss', warmup_loss, global_step=warmup_step)
                warmup_step += 1
        
#         if warmup_step%5==0:
#             torch.save(classifier.state_dict(), 
#                        Configs['checkpoint']['checkpoint_dir']+'inte_transformer_params_last.pth')
        if warmup_loss < min_loss:
            torch.save(classifier.state_dict(),
                       Configs['checkpoint']['checkpoint_dir']+'inte_transformer_params_best.pth')
            min_loss = warmup_loss
        
    
    ### train
#     step = 0
#     epochs = Configs['train']['n_epochs']
#     for epoch in range(epochs):
#         # Training loop
#         poly_lr_scheduler(optimizer, init_lr=Configs['optimizer']['init_lr'], iter=epoch, max_iter=epochs)
#         for batch_index, (data,target,_) in enumerate(train_loader, 0):
#             data, target = data.to('cuda'), target.to('cuda')
#             y = classifier(data)
#     #         logger.debug(f"y size:{y.shape}, tatget size{target.shape}")
#             train_loss = criterion(y, target)
#             optimizer.zero_grad()
#             train_loss.backward()
#             optimizer.step()
#     #         logger.info(f'Epoch: {epoch+1}, Train Loss: {train_loss}')
#             logger.info(f"Step: {step}, training Loss: {train_loss}")
#             writer.add_scalar('Training Loss', train_loss, global_step=step)
#             step += 1
        
#         if epoch%5==0:
#             val_loss = 0
#             correct = 0
#             total = 0
#             accuracy = 0
#             with torch.no_grad():
#                 for batch_index, (data,target,_) in enumerate(eval_loader, 0):
#                     data, target = data.to('cuda'), target.to('cuda')
#                     outputs = classifier(data)
#                     loss = criterion(outputs, target)
#                     val_loss += loss.item()
#                     _, predicted = torch.max(outputs, 1)
#                     total += target.size(0)  # Total number of samples
#                     correct += (predicted == target).sum().item()  # Count correct predictions

#             val_loss /= len(eval_loader)
#             accuracy = 100 * correct / total
#             writer.add_scalar('Validation Loss', val_loss, global_step=step)
#             writer.add_scalar('Validation Accuracy', accuracy, global_step=step)
#             logger.info(f'Epoch: {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

#         torch.save(classifier.state_dict(), 
#                    Configs['checkpoint']['checkpoint_dir']+'inte_transformer_params_latest.pth')
#         if train_loss < min_loss:
#             torch.save(classifier.state_dict(),
#                        Configs['checkpoint']['checkpoint_dir']+'inte_transformer_params_best.pth')
#             min_loss = train_loss

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("config_file", metavar="FILE", help="config file")
# parser.add_argument('--run-dir', metavar='DIR', help='run directory')
# parser.add_argument('--pdb', action='store_true', help='pdb')
args = parser.parse_args(args=['configs/eeg_torch.yml'])
# args, opts = parser.parse_known_args()
# f = 'configs/eeg_pt.yml'
with open(args.config_file, 'r') as file:
    configs = yaml.safe_load(file)
    
# configs['optimizer']['init_lr']
# train(Configs=configs)

In [ ]:
n_channels = 19
ae_1 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                               stride=3, kernel_size=7, dilation=1, groups=n_channels,
                               padding_mode='reflect'),
                     nn.BatchNorm1d(n_channels),
                     nn.ReLU())
ae_2 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                               stride=2, kernel_size=4, dilation=1, groups=n_channels,
                               padding_mode='reflect'),
                     nn.BatchNorm1d(n_channels),
                     nn.ReLU())
ae_3 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                               stride=2, kernel_size=4, dilation=1, groups=n_channels,
                               padding_mode='reflect'),
                     nn.BatchNorm1d(n_channels),
                     nn.ReLU())

# max_pool = nn.MaxPool1d(kernel_size=2, stride=1)
in_data = torch.randn(20, 19, 6000)
out_1 = ae_1(in_data)
out_2 = ae_2(out_1)
out_3 = ae_3(out_2)
# out_data = max_pool(out_data)
out_1.shape, out_2.shape, out_3.shape
# 5660 stride 11

In [ ]:
class transformer_classifier(nn.Module):
    def __init__(self, input_size:int, n_channels:int, model_hyp:dict, classes:int):
        super(transformer_classifier, self).__init__()
        
        self.ae_1 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                                     stride=3, kernel_size=7, dilation=1, groups=n_channels,
                                            padding_mode='reflect'),
                                 nn.BatchNorm1d(n_channels),
                                 nn.ReLU())
        
        self.ae_2 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                                     stride=2, kernel_size=4, dilation=1, groups=n_channels,
                                            padding_mode='reflect'),
                                 nn.BatchNorm1d(n_channels),
                                 nn.ReLU())

        self.ae_3 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                                     stride=2, kernel_size=4, dilation=1, groups=n_channels,
                                            padding_mode='reflect'),
                                 nn.BatchNorm1d(n_channels),
                                 nn.ReLU())
#         self.hidden_size = 597    # need to be calculated every time if you change shape of input
#         self.ae = AutoEncoder(input_size=input_size, hidden_size=model_hyp['d_model'])  
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=model_hyp['d_model'],
                                                        nhead=model_hyp['n_head'])
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=model_hyp['n_layer'])
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(model_hyp['d_model']*n_channels, classes)

    def forward(self, x):
        z = self.ae_1(x)
        z = self.ae_2(z)
        z = self.ae_3(z)
#         z = z[:, :, :1496] 
        logger.debug(f"ae output size: %{z.shape}")
        z = self.transformer_encoder(z)
        logger.debug(f"transformer output size: %{z.shape}")
        z = self.flatten(z)
        logger.debug(f"flatten output size: %{z.shape}")
        y = self.linear(z)
        return y

In [ ]:
in_data = torch.randn(20, 19, 6000).to('cuda')
classifier = transformer_classifier(input_size=6000, n_channels =19,model_hyp=configs['model'],classes=len(configs['dataset']['classes'])).to('cuda')
out = classifier(in_data)

In [2]:
in_data = torch.randn(20, 19, 6000).to('cuda')
network = resnet18().to('cuda')

In [3]:
output = network(in_data)

/home/yossi/anaconda3/envs/python38/lib/python3.8/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x96256 and 512x2)